# Rank 1 Constraint System overview

A Rank 1 Constraint System (R1CS) is an arithmetic circuit with the requirement that each **equality constraint** has one multiplication (and no restriction on the number of additions). This makes the representation of the arithmetic circuit compatible with the use of bilinear pairings.  
The output of a pairing $G_1 \cdot G_2 \to G_T$ cannot be paired again, as an element in cannot be used as part of the input of another pairing. Hence, we only allow one multiplication per constraint.

## The witness vector
In a Rank 1 Constraint System the witness vector is a $1 \times n$ vector that contains the value of all the input variables, the output variable, and the intermediate values. It shows you have executed the circuit from start to finish, knowing both the input, output, and all the intermediate values.  
E.g.:  
$$z = x^2y$$
If we know the solution to this we must know $x, y, z$. However as per R1CS, only one multiplication is allowed per contraint so we rewrite the above as
$$v_1 = xx$$
$$z = v_1y$$
Now, a *witness* means we not only know $x, y, z$ but also intermediate values i.e. $v_1$. So the vector will be :
$$[1, z, x, y, v_1]$$
one example is
$$[1, 18,3, 2, 9]$$

## Transforming into R1CS
We want out contraints to be of the form `result = left_hand_side x right_hand_side`
$$\underbrace{z}_\text{result} = \underbrace{x}_\text{left hand side} \times \underbrace{y}_\text{right hand side}$$
In the end our goal is to create system of equations of form:
$$\mathbf{O}\mathbf{a} = \mathbf{L}\mathbf{a}\circ\mathbf{R}\mathbf{a}$$
Where $O, L, R$ are matrices of size $n \times m$ and $a$ is the witness vector. 
Here **n = number of rows = number of conostraints in the circuits** and **m = number of columns in witness vector**.  

E.g:
$$\mathbf{O}\mathbf{a} = \mathbf{L}\mathbf{a}\circ\mathbf{R}\mathbf{a}$$
$$\underbrace{\begin{bmatrix}
0 & 1 & 0 & 0 \\
\end{bmatrix}}_{\mathbf{O}}\mathbf{a} =
\underbrace{\begin{bmatrix}
0 & 0 & 1 & 0 \\
\end{bmatrix}}_{\mathbf{L}}\mathbf{a}\circ
\underbrace{\begin{bmatrix}
0 & 0 & 0 & 1 \\
\end{bmatrix}}_{\mathbf{R}}\mathbf{a}$$
$$\begin{bmatrix}
0 & 1 & 0 & 0 \\
\end{bmatrix}
\begin{bmatrix}
1 \\
4223 \\
41 \\
103 \\
\end{bmatrix}=
\begin{bmatrix}
0 & 0 & 1 & 0 \\
\end{bmatrix}
\begin{bmatrix}
1 \\
4223 \\
41 \\
103 \\
\end{bmatrix}\circ
\begin{bmatrix}
0 & 0 & 0 & 1 \\
\end{bmatrix}
\begin{bmatrix}
1 \\
4223 \\
41 \\
103 \\
\end{bmatrix}$$


In [2]:
# The same above equation can be verified in python using numpy
import numpy as np

# define the matrices
O = np.matrix([[0,1,0,0]])
L = np.matrix([[0,0,1,0]])
R = np.matrix([[0,0,0,1]])

# witness vector
a = np.array([1, 4223, 41, 103])

# Multiplication `*` is element-wise, not matrix multiplication.
# Result contains a bool indicating an element-wise indicator that the equality is true for that element.
result = np.matmul(O, a) == np.matmul(L, a) * np.matmul(R, a) 

# check that every element-wise equality is true
assert result.all(), "result contains an inequality"

### Transforming r = x y z * u
As we can only have one multiply per constraint we can break above equality into:
$$ v_1 = x*y $$
$$ v_2 = z * u$$
$$r = v_1 * v_2$$
OR
$$ v_1 = x*y $$
$$ v_2 = v_1 * z$$
$$r = v_2 * u $$
Both are valid, but we will use the 1st one.

#### Size of $L, R\ and\ O$
We have 7 variables and 8th is a constant (remember 1st element of witness vector is always 1). Hence we will have 8 columns like
$$[1, r, x, y, z, u, v_1, v_2]$$
Hence, our matrices will have 8 columns. As we have 3 equality contraint, we will have *3 rows*.  

Now transforming into $O = L \times R$ we have

$$\underbrace{
\begin{matrix}
v_1 \\
v_2 \\
r \\
\end{matrix}
}_\text{ Output Terms }
\begin{matrix}
=\\
=\\
=
\end{matrix}
\underbrace{
\begin{matrix}
x \\
z \\
v_1 \\
\end{matrix}
}_\text{ Left Hand Terms }
\begin{matrix}
\times\\
\times\\
\times
\end{matrix}
\underbrace{
\begin{matrix}
y \\
u \\
v_2 \\
\end{matrix}
}_\text{ Right Hand Terms }$$

#### Rows of $L, R, O$
As per the the above, in the 1st row of $L$, we have the variable $x$ so we set that value to $1$ and rest to zero. Similarly, for row 2, we only have $z$, so we set that to column element to $1$ and others to 0 and so on. 
$$
\begin{matrix}
v_1 \\
v_2 \\
r \\
\end{matrix}
\begin{matrix}
=\\
=\\
=
\end{matrix}
\underset{\mathbf{L}}{\boxed{
\begin{matrix}
\color{red}{x} \\
\color{green}{z} \\
\color{violet}{v_1} \\
\end{matrix}
}}
\begin{matrix}
\times\\
\times\\
\times
\end{matrix}
\begin{matrix}
y \\
u \\
v_2 \\
\end{matrix}
\space\space\space\space
\begin{array}{c}
\begin{array}{cc}
\begin{matrix}
1 & r & x & y & z & u & v_1 & v_2 \\
\end{matrix}
\end{array} \\[10pt]
\begin{array}{cc}
\begin{bmatrix}
0 & 0 & \color{red}{1} & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & \color{green}{1} & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & \color{violet}{1} & 0 \\
\end{bmatrix}
\end{array}
\end{array}$$

Similarly we construct $R$ and $O$. And we can verify the same using numpy in python (see below cell).
$$\begin{matrix}
v_1 \\
v_2 \\
r \\
\end{matrix}
\begin{matrix}
=\\
=\\
=
\end{matrix}\begin{matrix}
x \\
z \\
v_1 \\
\end{matrix}\begin{matrix}
\times\\
\times\\
\times
\end{matrix}
\underset{\mathbf{R}}
{\boxed{
\begin{matrix}
\color{red}{y} \\
\color{green}{u} \\
\color{violet}{v_2} \\
\end{matrix}
}}
\space\space\space\space
\begin{array}{c}
\begin{array}{cc}
\begin{matrix}
1 & r & x & y & z & u & v_1 & v_2 \\
\end{matrix}
\end{array} \\[10pt]
\begin{array}{cc}
\begin{bmatrix}
0 & 0 & 0 & \color{red}{1} & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & \color{green}{1} & 0 & 0  \\
0 & 0 & 0 & 0 & 0 & 0 & 0 &\color{violet}{1}  \\
\end{bmatrix}
\end{array}
\end{array}$$
$$\mathbf{O}=\begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1  \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0  \\
\end{bmatrix}$$

In [4]:
# enter the A B and C from above
L = np.matrix([[0,0,1,0,0,0,0,0],
              [0,0,0,0,1,0,0,0],
              [0,0,0,0,0,0,1,0]])

R = np.matrix([[0,0,0,1,0,0,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,0,1]])

O = np.matrix([[0,0,0,0,0,0,1,0],
              [0,0,0,0,0,0,0,1],
              [0,1,0,0,0,0,0,0]])

# random values for x, y, z, and u
import random
x = random.randint(1,1000)
y = random.randint(1,1000)
z = random.randint(1,1000)
u = random.randint(1,1000)

# compute the algebraic circuit
r = x * y * z * u
v1 = x*y
v2 = z*u

# create the witness vector
a = np.array([1, r, x, y, z, u, v1, v2])

# element-wise multiplication, not matrix multiplication
result = np.matmul(O, a) == np.multiply(np.matmul(L, a), np.matmul(R, a))

assert result.all(), "system contains an inequality"

Whenever there are additive constants, we simply place them in the column, which by convention is the first column.
$$z = x * y + 2$$
Can bwe written as below to so it of the form $O = L \times R$
$$-2 + z = xy$$
$$\mathbf{L} = \begin{bmatrix}
0 & 0 & 1 & 0 \\
\end{bmatrix}$$
$$\mathbf{R} = \begin{bmatrix}
0 & 0 & 0 & 1 \\
\end{bmatrix}$$
$$\mathbf{O} = \begin{bmatrix}
-2 & 1 & 0 & 0 \\
\end{bmatrix}$$

### A large constraint
$$z = 3x^2y + 5xy – x – 2y + 3$$
We will break it up as:
$$\begin{align*}
v_1 &= 3xx \\
v_2 &= v_1y \\
-v_2 + x + 2y – 3 + z  &= 5xy \\
\end{align*} $$
Given that our witness vector is of the form
$$[1, z, x, y, v_1, v_2]$$
out matrices will have **3 rows** as there are 3 equality contraints and **6 columns** are there are 6 variables for the witness.
$$\begin{align*}
\color{red}{v_1} &= \color{green}{3x}\color{violet}{x} \\
\color{red}{v_2} &= \color{green}{v_1}\color{violet}{y} \\
\color{red}{-v_2 + x + 2y – 3 + z}  &= \color{green}{5x}\color{violet}{y} \\
\end{align*}$$
Evidently the $L, R, O$ matrices will be:
$$L = \begin{bmatrix}
0 & 0 & \textcolor{green}{3} & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & \textcolor{green}{1} & 0 \\
0 & 0 & \textcolor{green}{5} & 0 & 0 & 0 \\
\end{bmatrix}$$
$$R = \begin{bmatrix}
0 & 0 & \textcolor{violet}{1} & 0 & 0 & 0 \\
0 & 0 & 0 & \textcolor{violet}{1} & 0 & 0 \\
0 & 0 & 0 & \textcolor{violet}{1} & 0 & 0 \\
\end{bmatrix}$$
$$O = \begin{bmatrix}0 & 0 & 0 & 0 & \textcolor{red}{1} & 0 \\
0 & 0 & 0 & 0 & 0 & \textcolor{red}{1} \\
\textcolor{red}{-3} & \textcolor{red}{1} & \textcolor{red}{1} & \textcolor{red}{2} & 0 & \textcolor{red}{-1} \\
\end{bmatrix}$$


In [5]:
import numpy as np
import random

# Define the matrices
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,5,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,1,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = random.randint(1,1000)
y = random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x - 2 * y + 3 # the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))
assert result.all(), "result contains an inequality"

Everything is done modulo prime in R1CS. Mostly we will have a set of arithmatics constriants (till now we only used one equation). In such case, we can combine equation or we may need to split equations based on number of "true" multiplication in each equality.

## Circom 
In Circom, *a language for constructing Rank 1 Constraint Systems*, the finite field uses the prime number:
$$21888242871839275222246405745257275088548364400416034343698204186575808495617$$
Example code for $z = x\ast y \ast z \ast u$:

```circom
pragma circom 2.0.8;

template Multiply4() {
    signal input x;
    signal input y;
    signal input z;
    signal input u;

    signal v1;
    signal v2;

    signal out;

    v1 <== x * y;
    v2 <== z * u;

    out <== v1 * v2;
}

template main = Multiply4();
```

![image](https://www.rareskills.io/wp-content/uploads/2024/08/935a00_21b46f6f9ffd4a80b1a2a374be0de279~mv2.png)

# Building a Zero Knowledge Proof from an R1CS

A zero knowledge proof for an R1CS is accomplished by converting the witness vector into [finite field elliptic curve points](https://www.rareskills.io/post/elliptic-curves-finite-fields) and replacing the Hadamard product with a [bilinear pairing](https://www.rareskills.io/post/bilinear-pairing) for each row.  
Given a Rank 1 Constraint System where each matrix has rows and columns, we write it as:
$$\mathbf{L}\mathbf{a}\circ\mathbf{R}\mathbf{a}=\mathbf{O}\mathbf{a}$$
In expanded form, it looks like
$$\left[ \begin{array}{ccc}
l_{1,1} & \cdots & l_{1,m} \\
\vdots & \ddots & \vdots \\
l_{n,1} & \cdots & l_{n,m}
\end{array} \right]
\left[ \begin{array}{c}
a_1 \\
\vdots \\
a_m
\end{array} \right]
\circ
\left[ \begin{array}{ccc}
r_{1,1} & \cdots & r_{1,m} \\
\vdots & \ddots & \vdots \\
r_{n,1} & \cdots & r_{n,m}
\end{array} \right]
\left[ \begin{array}{c}
a_1 \\
\vdots \\
a_m
\end{array} \right]
=
\left[ \begin{array}{ccc}
o_{1,1} & \cdots & o_{1,m} \\
\vdots & \ddots & \vdots \\
o_{n,1} & \cdots & o_{n,m}
\end{array} \right]
\left[ \begin{array}{c}
a_1 \\
\vdots \\
a_m
\end{array} \right]$$
Which is simplified to 
$$=
\begin{array}{ccc}
\sum_{i=1}^m a_i l_{1,i}  \sum_{i=1}^m a_i r_{1,i} = \sum_{i=1}^m a_i o_{1,i} \\
\sum_{i=1}^m a_i l_{2,i}  \sum_{i=1}^m a_i r_{2,i} = \sum_{i=1}^m a_i o_{2,i} \\
\vdots \\
\sum_{i=1}^m a_i l_{n,i}  \sum_{i=1}^m a_i r_{n,i} = \sum_{i=1}^m a_i o_{n,i}
\end{array}$$
In this setup, we can prove to a verifier that we have a witness vector that satisfies the R1CS simply by giving them the vector , but with the obvious drawback that this is not a zero knowledge proof!  

## Zero knowledge proof algorithm for an R1CS.
If we “encrypt” the witness vector by multiplying each entry with or , the math will still work properly! For example:
$$\begin{bmatrix}
1 & 2 \\
3 & 4 \\
\end{bmatrix}
\begin{bmatrix}
4 \\
5
\end{bmatrix}
= \begin{bmatrix}
14 \\
32
\end{bmatrix}$$
Is equivalent to 
$$\begin{bmatrix}
1 & 2 \\
3 & 4 \\
\end{bmatrix}
\begin{bmatrix}
4G_1 \\
5G_1
\end{bmatrix}
= \begin{bmatrix}
14G_1 \\
32G_1
\end{bmatrix}$$
In other words, each time we multiply the column vector by a row in the square matrix, we carry out two elliptic curve point multiplications and one elliptic curve addition.

## Notation for elliptic curves

We say $[aG_1]_1$ is a $\mathbb{G}_1$ elliptic curve point created from multiplying the field element $a$ by $G_1$ . We say $[aG_2]_2$ is a $\mathbb{G}_2$ elliptic curve point generated by multiplying $a$ with the generator $G_2$. Because of the discrete log problem, we cannot extract $a$ given $[aG_1]_1$ or $[aG_2]_2$ . Given a $A \in \mathbb{G}_1$ and $B \in \mathbb{G}_2$ point, we say the pairing of the two points is $A\bullet B$.  
So The $L$ and $R$ matrices become:
$$ L =
\left[ \begin{array}{c}
\sum_{i=1}^m l_{1,i}[a_i G_1]_1 \\
\sum_{i=1}^m l_{2,i}[a_i G_1]_1 \\
\vdots \\
\sum_{i=1}^m l_{n,i}[a_i G_1]_1
\end{array} \right]$$
$$R =
\left[ \begin{array}{c}
\sum_{i=1}^m r_{1,i}[a_i G_2]_2 \\
\sum_{i=1}^m r_{2,i}[a_i G_2]_2 \\
\vdots \\
\sum_{i=1}^m r_{n,i}[a_i G_2]_2
\end{array} \right]$$

We give the encrypted $L$ and $R$ and $Oa$ encrypted in $G_1$ vectors to the verifier who does the verification step as follows:
$$\left[ \begin{array}{c}
\sum_{i=1}^m l_{i,1}[a_i G_1]_1 \\
\sum_{i=1}^m l_{i,1}[a_i G_1]_1 \\
\vdots \\
\sum_{i=1}^m l_{i,1}[a_i G_1]_1
\end{array} \right]
\begin{matrix}
\bullet \\
\bullet \\
\vdots \\
\bullet
\end{matrix}
\left[ \begin{array}{c}
\sum_{i=1}^m r_{i,1}[a_i G_2]_2 \\
\sum_{i=1}^m r_{i,1}[a_i G_2]_2 \\
\vdots \\
\sum_{i=1}^m r_{i,1}[a_i G_2]_2
\end{array} \right]\stackrel{?}{=}
\left[ \begin{array}{c}
\sum_{i=1}^m o_{i,1}[a_i G_1]_1 \\
\sum_{i=1}^m o_{i,1}[a_i G_1]_1 \\
\vdots \\
\sum_{i=1}^m o_{i,1}[a_i G_1]_1
\end{array} \right]
\begin{matrix}
\bullet \\
\bullet \\
\vdots \\
\bullet
\end{matrix}
\left[ \begin{array}{c}
G_2 \\
G_2 \\
\vdots \\
G_2
\end{array} \right]$$
$$=
\begin{array}{c}
\sum_{i=1}^m l_{i,1}[a_i G_1]_1\bullet \sum_{i=1}^m r_{i,1}[a_i G_2]_2 \\
\sum_{i=1}^m l_{i,2}[a_i G_1]_1\bullet \sum_{i=1}^m r_{i,2}[a_i G_2]_2  \\
\vdots \\
\sum_{i=1}^m l_{i,n}[a_i G_1]_1\bullet \sum_{i=1}^m r_{i,n}[a_i G_2]_2
\end{array}
\stackrel{?}{=}
\begin{array}{c}
\sum_{i=1}^m o_{i,1}[a_i G_1]_1\bullet G_2 \\
\sum_{i=1}^m o_{i,2}[a_i G_1]_1\bullet G_2 \\
\vdots \\
\sum_{i=1}^m o_{i,n}[a_i G_1]_1 \bullet G_2
\end{array}$$
Note that the verifier does the extra step of multiplying given $Oa$ with $G_2$ to obtain a point in $G_{12}$ which is then compared with $pairing(La, Ra)$.

**NOTE**:*If we need some parts of witness vector to be public like $[1,y]$ in $x³ + 5x + 5 = y$ then we don't encrypt those public inputs and leave it to the verifier to encrypt them.  

## Dealing with a malicious prover.
As a verifier, to *confirm* that the discrete log of encrypted $L$ and $R$ is indeed the same ($a$ in our case i.e the witness vector), the verifier can compare the equality by pairing both vectors of points with a vector of the opposite generator and seeing that the resulting $\mathbb{G}_{12}$ points are equal
$$\begin{bmatrix}
a_1G_1 \\
a_2G_1 \\
\vdots \\
a_mG_1
\end{bmatrix}
\begin{matrix}
\bullet \\
\bullet \\
\vdots \\
\bullet
\end{matrix}
\begin{bmatrix}
G_2 \\
G_2 \\
\vdots \\
G_2
\end{bmatrix}
\stackrel{?}{=}
\begin{bmatrix}
a_1G_2 \\
a_2G_2 \\
\vdots \\
a_mG_2
\end{bmatrix}
\begin{matrix}
\bullet \\
\bullet \\
\vdots \\
\bullet
\end{matrix}
\begin{bmatrix}
G_1 \\
G_1 \\
\vdots \\
G_1
\end{bmatrix}$$

**Note**: Here although we are encrypting $a$, it is not entirely impossible for a hacker to guess the $a$. Once guessed, they can verify their guess by comparing $L$ and $R$. Groth16 deals with this shortcoming. Also, this ZK is R1CS is ineffecient for real life examples and is never used

In [1]:
# Implementation of one of the problem (directly above this in python) as per above understanding

import numpy as np
import random
from py_ecc.bn128 import eq, pairing, multiply, G1, G2

# Define the matrices
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,5,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,1,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = random.randint(1,1000)
y = random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x - 2 * y + 3 # the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])


In [15]:
# Executing the below may take some time. wait for it to finish!
enc_L = []
for scalar in L.dot(w):
    enc_L.append(multiply(G1, scalar))
    # print(enc_L[-1])

enc_R = []
for scalar in R.dot(w):
    enc_R.append(multiply(G2, scalar))
    # print(enc_R[-1])

enc_O = []
for scalar in O.dot(w):
    enc_O.append(multiply(G1, scalar))
    # print(enc_O[-1])

print(len(enc_L), len(enc_R), len(enc_O))
result = [
    eq(pairing(two[1], two[0]), pairing(G2, one)) for two, one in zip(zip(enc_L, enc_R), enc_O)
]

print(result)

# result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))
# assert result.all(), "result contains an inequality"

3 3 3
[True, True, True]
